<h1> Binary Text Classification

upload kemudian extract berkas zip dataset yelp

In [1]:
import zipfile, os
local_zip = 'DatasetYelp.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/yelp')
zip_ref.close()

Setelah dataset diunduh, kita load dataset pada Colaboratory. Dan pastinya kita buat dataframe dari dataset kita agar lebih mudah diproses.

In [2]:
import pandas as pd
df = pd.read_csv('/yelp/sentiment labelled sentences/yelp_labelled.txt', names=['sentence', 'label'], sep='\t')

In [3]:
df.tail()

,sentence,label
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0
999,"Then, as if I hadn't wasted enough of my life ...",0


Setelah itu kita bagi dataset menjadi train dan test set.

In [4]:
from sklearn.model_selection import train_test_split
kalimat = df['sentence'].values
y = df['label'].values
kalimat_latih, kalimat_test, y_latih, y_test = train_test_split(kalimat, y, test_size=0.2)

Agar teks dapat dipahami oleh model, kita harus lakukan tokenisasi. Gunakan fungsi tokenizer pada data latih dan data test. Jangan lupa gunakan fungsi pad_sequences agar setiap sequence sama panjang.

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=250, oov_token='x')
tokenizer.fit_on_texts(kalimat_latih) 
tokenizer.fit_on_texts(kalimat_test)
 
sekuens_latih = tokenizer.texts_to_sequences(kalimat_latih)
sekuens_test = tokenizer.texts_to_sequences(kalimat_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

Untuk arsitektur yang akan digunakan adalah layer embedding, dengan argumen pertama sesuai dengan jumlah vocabulary/kata yang kita pakai pada tokenizer. Argumen selanjutnya adalah dimensi embedding, dan input_length yang merupakan panjang dari sequence. Nah di kita tidak menggunakan layer Flatten melainkan kita menggantinya dengan GlobalAveragePooling1D. Fungsi ini bekerja lebih baik pada kasus NLP dibanding Flatten.

In [8]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(250, 16),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


#compile model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Setelah arsitektur model dibentuk, dan loss function serta optimizer ditentukan, kita dapat memulai pelatihan model kita. 

In [9]:
num_epochs = 30
history = model.fit(padded_latih, y_latih, epochs=num_epochs, 
                    validation_data=(padded_test, y_test), verbose=2)

Epoch 1/30
25/25 - 3s - loss: 0.6929 - accuracy: 0.5213 - val_loss: 0.6921 - val_accuracy: 0.5750 - 3s/epoch - 116ms/step
Epoch 2/30
25/25 - 0s - loss: 0.6910 - accuracy: 0.6075 - val_loss: 0.6907 - val_accuracy: 0.5650 - 88ms/epoch - 4ms/step
Epoch 3/30
25/25 - 0s - loss: 0.6882 - accuracy: 0.6250 - val_loss: 0.6881 - val_accuracy: 0.5850 - 93ms/epoch - 4ms/step
Epoch 4/30
25/25 - 0s - loss: 0.6839 - accuracy: 0.6475 - val_loss: 0.6838 - val_accuracy: 0.6350 - 88ms/epoch - 4ms/step
Epoch 5/30
25/25 - 0s - loss: 0.6775 - accuracy: 0.6600 - val_loss: 0.6778 - val_accuracy: 0.6400 - 124ms/epoch - 5ms/step
Epoch 6/30
25/25 - 0s - loss: 0.6677 - accuracy: 0.6862 - val_loss: 0.6693 - val_accuracy: 0.6300 - 91ms/epoch - 4ms/step
Epoch 7/30
25/25 - 0s - loss: 0.6540 - accuracy: 0.6988 - val_loss: 0.6576 - val_accuracy: 0.6650 - 110ms/epoch - 4ms/step
Epoch 8/30
25/25 - 0s - loss: 0.6368 - accuracy: 0.7200 - val_loss: 0.6460 - val_accuracy: 0.6450 - 100ms/epoch - 4ms/step
Epoch 9/30
25/25 - 0s